In [1]:
library(brms) #predictability: Double hierearchical model
library(MCMCglmm) #HPDinterval
library(dplyr) #%>%
library(lmerTest) #lmer
#library(MuMIn) #r2
#library(merTools) #sim
library(sjPlot) #plot coeff
library(ggplot2) #ylim in plot
library(rstan) #model diagnostic

Warning message:
"le package 'brms' a été compilé avec la version R 4.1.2"
Le chargement a nécessité le package : Rcpp

Warning message:
"le package 'Rcpp' a été compilé avec la version R 4.1.2"
Loading 'brms' package (version 2.16.3). Useful instructions
can be found by typing help('brms'). A more detailed introduction
to the package is available through vignette('brms_overview').


Attachement du package : 'brms'


L'objet suivant est masqué depuis 'package:stats':

    ar


Warning message:
"le package 'MCMCglmm' a été compilé avec la version R 4.1.2"
Le chargement a nécessité le package : Matrix

Warning message:
"le package 'Matrix' a été compilé avec la version R 4.1.2"
Le chargement a nécessité le package : coda

Warning message:
"le package 'coda' a été compilé avec la version R 4.1.2"
Le chargement a nécessité le package : ape

Warning message:
"le package 'ape' a été compilé avec la version R 4.1.2"

Attachement du package : 'MCMCglmm'


L'objet suivant est masqué depuis 'pac

In [2]:
library(ggplot2)

In [3]:
library(parallel)
my.cores = detectCores()-1 #minus 1 to avoid bocking the computer
my.cores

[1] 11

In [4]:
path_ = 'G:/VPHI/Welfare/2- Research Projects/OFHE2.OriginsE2/DataOutput/TrackingSystem/ALLDATA_'
#df = read.csv(file.path(path_,'df_daily_aggregatedHA.csv'), header = TRUE, sep = ",")
df = read.csv(file.path(path_,'OFH_df_MVT_ALL.csv'), header = TRUE, sep = ",")

df$HenID = as.factor(df$HenID)   
df$PenID = as.factor(df$PenID)
df$level = as.factor(df$level) 
df$CLASS = as.factor(df$CLASS) 
df$Treatment = as.factor(df$Treatment)
df$time = scale(df$WIB, center=FALSE, scale=sd(df$WIB, na.rm = TRUE)) #as not centering, we need to add this
df$time2_ = poly(df$time, degree=2,raw=TRUE)[,2]
df$time2 = scale(df$time2_, center=FALSE, scale=sd(df$time2_, na.rm = TRUE))
#df$KBF_interp = scale(df$KBF_interp, center=TRUE)
#df$FeatherDamage_interp = scale(df$FeatherDamage_interp, center=TRUE)
#df$weight_interp = scale(df$weight_interp, center=TRUE)

#scale: mean-centering of the environmental variable so that intercepts reflect average values for the HenID and ind. 
#df$temperature_C_avg_scale = scale(df$temperature_C_avg, center=TRUE, scale=TRUE)

#setting reference group
df <- df %>% mutate(Treatment = relevel(Treatment, ref = "TRAN"))
contrasts(df$Treatment)
df <- df %>% mutate(CLASS = relevel(CLASS, ref = "REXP"))
contrasts(df$CLASS)
print(dim(df))
summary(df)
write.csv(df, file.path(path_,'OFH__df_MVT_ALL_time.csv'))

,OFH
TRAN,0
OFH,1


,LEXP,MEXP
REXP,0,0
LEXP,1,0
MEXP,0,1


[1] 5875   19


     HenID          PenID             level      Treatment    CLASS     
 hen_10 :  39   pen10  : 769   2021-01-10: 159   TRAN:2920   REXP:3397  
 hen_112:  39   pen11  : 766   2021-01-17: 159   OFH :2955   LEXP:1110  
 hen_118:  39   pen4   : 738   2021-02-07: 159               MEXP:1368  
 hen_134:  39   pen5   : 736   2021-04-18: 159                          
 hen_139:  39   pen9   : 733   2021-01-24: 158                          
 hen_145:  39   pen12  : 728   2021-02-28: 158                          
 (Other):5641   (Other):1405   (Other)   :4923                          
      DOA             WOA             DIB             WIB       
 Min.   :123.0   Min.   :18.00   Min.   :  5.0   Min.   : 1.00  
 1st Qu.:193.0   1st Qu.:28.00   1st Qu.: 75.0   1st Qu.:11.00  
 Median :256.0   Median :37.00   Median :138.0   Median :20.00  
 Mean   :261.7   Mean   :37.81   Mean   :143.7   Mean   :20.81  
 3rd Qu.:333.0   3rd Qu.:48.00   3rd Qu.:215.0   3rd Qu.:31.00  
 Max.   :410.0   Max.   :5

In [5]:
sd(df$time2)
sd(df$time2_)
sd(df$time)

[1] 1

[1] 3.744731

[1] 1

In [6]:
#df$Matrix_proportions = data.matrix(df[,c('propindoor_duration_2_Zone','propindoor_duration_3_Zone', 'propindoor_duration_4_Zone', 'propindoor_duration_5_Zone')])

In [ ]:
iter_ = 10000*10
iter_wu = 8000*10
bind <- function(...) cbind(...) #cbind is reserve for multivariate in brms!
#put 4 as reference!
frml = bf(bind(propindoor_duration_4_Zone, propindoor_duration_2_Zone, propindoor_duration_3_Zone,
               propindoor_duration_5_Zone) ~ time + time2 + Treatment + time:Treatment + PenID + CLASS + scale(KBF_interp) + scale(FeatherDamage_interp) + scale(weight_interp) + (1|HenID))
fit_moreit = brm(frml, data = df, family = dirichlet(link = 'logit', link_phi = 'log'), 
                 iter = iter_, warmup = iter_wu, thin = 50, chains = 3, cores = my.cores, seed = 12345,
                 control = list(max_treedepth=11, adapt_delta=0.9))
summary(fit_moreit)
save(fit_moreit, file = file.path(path_, 'OFH_dirichlet_80birdsPenIDtimesquare_moreit_allfixedeffect_allhens.rda'))
#load(file.path(path_,"OFH_dirichlet_80birdsPenIDtimesquare_moreit.rda"))  #OLD: 10 divergent transitions
#load(file.path(path_,"OFH_dirichlet_80birdsPenIDtimesquare_moreit_allfixedeffect_allhens.rda"))  #OLD: 10 divergent transitions

Warning message:
"Rows containing NAs were excluded from the model."
Compiling Stan program...

Start sampling



In [ ]:
#as hens are in a unique pen:
li_dfpred = list()
li_dfnewdata = list()
bind <- function(...) cbind(...) #cbind is reserve for multivariate in brms!
for (penid in unique(df$PenID)){
    print(penid)
    df_newdata = expand.grid(Treatment=c("OFH",'TRAN'),
                             PenID = penid,
                             HenID = unique(df[df$PenID==penid,]$HenID), #unique(df$HenID), #sample(unique(df$HenID), 10),
                             time = unique(df$time),
                             time2 = unique(df$time2),
                             Treatment = unique(df[df$PenID==penid,]$Treatment))
    dim(df_newdata)
    #head(df_newdata, 2)
    #predict.MCMCglmm() : https://www.rdocumentation.org/packages/MCMCglmm/versions/2.32/topics/predict.MCMCglmm
    #credible intervals: https://stat.ethz.ch/pipermail/r-sig-mixed-models/2018q1/026408.html 
    df_pred = predict(fit_moreit, newdata=df_newdata, interval='confidence') #marginal=fit$Random$formula OR NULL?
    #dim(df_pred)
    #head(df_pred,3)
    ##li_dfpred = append(li_dfpred,list(df_pred))
    ##li_dfnewdata = append(li_dfnewdata,list(df_newdata))
    write.csv(df_pred, file.path(path_,paste0('OFH_',penid,'_df_pred_all_moreit_allfixedeffect_allhens.csv')))
    write.csv(df_newdata, file.path(path_,paste0('OFH_',penid,'_df_newdata_all_moreit_allfixedeffect_allhens.csv')))
}
##df_pred_all = bind_rows(li_dfpred)
##df_newdata_all = bind_rows(li_dfnewdata)
##dim(df_pred_all)
##dim(df_newdata_all)
##write.csv(df_pred_all, file.path(path_,paste0('OFH_',penid,'_df_pred_all_moreit.csv')))
##write.csv(df_newdata_all, file.path(path_,paste0('OFH_',penid,'_df_newdata_all_moreit.csv')))

In [ ]:
#TODO NEXT ROUND: all birds, account for CLASS; health, weight,

In [ ]:
#dirichlet diagnotic pp_check
#https://discourse.mc-stan.org/t/using-pp-check-for-dirichlet-model/8033/6
#good to know: http://paul-buerkner.github.io/brms/reference/conditional_effects.html
#to read: https://www.google.com/imgres?imgurl=https%3A%2F%2Fars.els-cdn.com%2Fcontent%2Fimage%2F1-s2.0-S0169207021000583-gr1.jpg&imgrefurl=https%3A%2F%2Fwww.sciencedirect.com%2Fscience%2Farticle%2Fpii%2FS0169207021000583&tbnid=ot_CAfIcu_8oPM&vet=12ahUKEwiTl-ez1tX4AhWyPOwKHSVVAhMQMygkegUIARDvAQ..i&docid=medWEoPnTXoAaM&w=531&h=525&q=plot%20Dirichlet%20prediction%20with%20brms%20odel&ved=2ahUKEwiTl-ez1tX4AhWyPOwKHSVVAhMQMygkegUIARDvAQ